<a href="https://colab.research.google.com/github/Weizhuo-Zhang/c-_primer_plus/blob/master/chapter08_function/chapter_08_function_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>